In [1]:
import gradio as gr
import os
demo =None

def start_game(main_loop, share=False):
    global demo
    
    if demo is not None:
        demo.close()
    
    demo = gr.ChatInterface(
        main_loop,
        chatbot=gr.Chatbot(height=250,placeholder="Type 'styart game' to begin"),
        textbox=gr.Textbox(placeholder="What do you do next?", container=False, scale=7),
        title="AI RPG",
        theme="soft",
        examples=["Look around","Continue the story"],
        cache_examples=False,
        retry_btn="Retry",
        undo_btn="Undo",
        clear_btn="Clear"
    )
    
    demo.launch(share=share,server_name="0.0.0.0")
    
# def test_main_loop(message, history):
#     return 'Enter Action: ' + message

# start_game(test_main_loop)

In [4]:
from helper import load_world, save_world
from together import Together
from helper import get_together_api_key, load_env

client = Together(api_key=get_together_api_key())

world = load_world('../shared_data/Kyropeia.json')
kingdom = world['kingdoms']['Eldrida']
town = kingdom['towns']['Luminaria']
character=town['npcs']['Elwyn Stormbringer']

In [5]:
system_prompt=""" You are an AI Game master. Your job is to create a
start to an adventure based on the world, king, town and character
a palyer is playing as.
Instructions:
You must only use 2-4 sentemces \
Write in second person. For example: "You are jack" \
Write in present tense. For example "You stand at..." \
First describe the character and their backstory. \
Then describes where they start and what they see around them."""

world_info = f"""
World: {world}
Kingdom: {kingdom}
Town: {town}
Your Character: {character}
"""

In [6]:
model_output = client.chat.completions.create(
     model="meta-llama/Llama-3-70b-chat-hf",
    temperature=1.0,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info + '\nYour Start:'}
    ],
)

In [7]:
model_output

ChatCompletionResponse(id='8ea009db6b3a156a-SJC', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1732856472, model='meta-llama/Llama-3-70b-chat-hf', choices=[ChatCompletionChoicesData(index=0, logprobs=None, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="You are Elwyn Stormbringer, a 25-year-old inventor with a passion for harnessing the power of the Colossi. You stand in the midst of Luminaria, a town of radiant beauty, surrounded by crystal formations that illuminate the streets and buildings. The Academy of Elemental Studies, where scholars from across Eldrida come to unlock the secrets of the Colossi, is just a short walk from your current location. You wear a pair of goggles on your forehead and carry a worn leather-bound journal filled with sketches of your inventive ideas. The weight of your past mistakes, including your mentor's failed experiment that destroyed a section of the

In [8]:
start = model_output.choices[0].message.content
print(start)
world['start'] = start
save_world(world, '../shared_data/YourWorld_L1.json')

You are Elwyn Stormbringer, a 25-year-old inventor with a passion for harnessing the power of the Colossi. You stand in the midst of Luminaria, a town of radiant beauty, surrounded by crystal formations that illuminate the streets and buildings. The Academy of Elemental Studies, where scholars from across Eldrida come to unlock the secrets of the Colossi, is just a short walk from your current location. You wear a pair of goggles on your forehead and carry a worn leather-bound journal filled with sketches of your inventive ideas. The weight of your past mistakes, including your mentor's failed experiment that destroyed a section of the city, drives you to create a machine that can prevent such disasters from occurring again.


In [9]:
world

{'name': 'Kyropeia',
 'description': 'Kyropeia is a realm where ancient, gargantuan creatures known as the "Colossi" roam the land, their massive bodies serving as the foundation for sprawling metropolises. These beasts, born from the earth and nurtured by the planet\'s elemental forces, have formed symbiotic relationships with the inhabitants of Kyropeia, who have learned to harness their power and build sprawling cities upon their backs. As the Colossi migrate across the landscape, the cities shift and change, adapting to the ever-changing environment.',
 'kingdoms': {'Eldrida': {'name': 'Eldrida',
   'description': 'Eldrida is a kingdom built upon the back of the oldest and wisest Colossus, Arkeia. The Eldridans are a scholarly people, fascinated by the ancient magic that flows through the Colossi. They have developed a deep understanding of the elemental forces, allowing them to harness the power of the land to fuel their innovative technology. Led by the enigmatic Queen Lyra, Eldr

In [15]:

def run_action(message, history, game_state):
    
    if(message == 'start game'):
        return game_state['start']

    system_prompt = """You are an AI Game master. Your job is to write what \
happens next in a player's adventure game.\
Instructions: \
You must on only write 1-3 sentences in response. \
Always write in second person present tense. \
Ex. (You look north and see...)"""
    
    world_info = f"""
World: {game_state['world']}
Kingdom: {game_state['kingdom']}
Town: {game_state['town']}
Your Character:  {game_state['character']}"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": world_info}
    ]
    for action in history:
        messages.append({"role": "assistant", "content": action[0]})
        messages.append({"role": "user", "content": action[1]})

    messages.append({"role": "user", "content": message})
    model_output = client.chat.completions.create(
        model="meta-llama/Llama-3-70b-chat-hf",
        messages=messages
    )
    
    result = model_output.choices[0].message.content
    return result

In [16]:
game_state = {
    "world": world['description'],
    "kingdom": kingdom['description'],
    "town": town['description'],
    "character": character['description'],
    "start": start,
}

def main_loop(message, history):
    return run_action(message, history, game_state)

In [17]:
game_state

{'world': 'Kyropeia is a realm where ancient, gargantuan creatures known as the "Colossi" roam the land, their massive bodies serving as the foundation for sprawling metropolises. These beasts, born from the earth and nurtured by the planet\'s elemental forces, have formed symbiotic relationships with the inhabitants of Kyropeia, who have learned to harness their power and build sprawling cities upon their backs. As the Colossi migrate across the landscape, the cities shift and change, adapting to the ever-changing environment.',
 'kingdom': 'Eldrida is a kingdom built upon the back of the oldest and wisest Colossus, Arkeia. The Eldridans are a scholarly people, fascinated by the ancient magic that flows through the Colossi. They have developed a deep understanding of the elemental forces, allowing them to harness the power of the land to fuel their innovative technology. Led by the enigmatic Queen Lyra, Eldrida is a beacon of knowledge and progress, attracting scholars and inventors f

In [13]:
def main_loop(message, history):
    return run_action(message, history, game_state)

In [18]:
start_game(main_loop, True)

Closing server running on port: 7860
Running on local URL:  https://0.0.0.0:7860
Running on public URL: https://956bd2c8dd40ad1ee3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


model_output=client.chat.completions.create()